<a href="https://colab.research.google.com/github/hood-boi/world-news-chatbot/blob/master/Seq2Seqipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
import torchtext
import random
import requests
import json
import time

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
text_field = torchtext.data.Field(sequential=True,      # text sequence
                                  tokenize=lambda x: x, # because are building a character-RNN
                                  include_lengths=True, # to track the length of sequences, for batching
                                  batch_first=True,
                                  use_vocab=True,
                                  init_token="<BOS>",
                                  eos_token="<EOS>"
                                 )       # to turn each character into an integer index
label_field = torchtext.data.Field(sequential=True,    # text sequence
                                   use_vocab=True,     # don't need to track vocabulary
                                   is_target=True,      
                                   batch_first=True,
                                   tokenize=lambda x: x,
                                   preprocessing=lambda x: x,
                                   init_token="<BOS>",
                                   eos_token="<EOS>"
                                  ) 

fields = [('reply', label_field), ('context', text_field)]
dataset1 = torchtext.data.TabularDataset("/content/gdrive/My Drive/Chatbot/rtodayilearned.txt", "tsv", fields); # name of the file
dataset2 = torchtext.data.TabularDataset("/content/gdrive/My Drive/Chatbot/rAskReddit.txt", "tsv", fields); # name of the file
dataset3 = torchtext.data.TabularDataset("/content/gdrive/My Drive/Chatbot/rnews.txt", "tsv", fields); # name of the file
dataset4 = torchtext.data.TabularDataset("/content/gdrive/My Drive/Chatbot/rworldnews.txt", "tsv", fields); # name of the file
dataset5 = torchtext.data.TabularDataset("/content/gdrive/My Drive/Chatbot/routoftheloop.txt", "tsv", fields); # name of the file
dataset6 = torchtext.data.TabularDataset("/content/gdrive/My Drive/Chatbot/rnottheonion.txt", "tsv", fields); # name of the file
dataset7 = torchtext.data.TabularDataset("/content/gdrive/My Drive/Chatbot/rCasualConversation.txt", "tsv", fields); # name of the file
dataset8 = torchtext.data.TabularDataset("/content/gdrive/My Drive/Chatbot/text.txt", "tsv", fields);
dataset = torch.utils.data.ConcatDataset([dataset1, dataset2, dataset3, dataset4, dataset5, dataset6, dataset7, dataset8])

In [0]:
! cat '/content/gdrive/My Drive/Chatbot/text.txt'

In [0]:
train = torchtext.data.Dataset(dataset, fields)
text_field.build_vocab(train)
label_field.build_vocab(train)
print(text_field.vocab.itos)
print(label_field.vocab.itos)
input_vocab_size = len(text_field.vocab.itos)
reply_vocab_size = len(label_field.vocab.itos)
print("Input Vocab Size: ", input_vocab_size)
print("Reply Vocab size: ", reply_vocab_size)

['<unk>', '<pad>', '<BOS>', '<EOS>', ' ', 'e', 't', 'a', 'o', 'i', 'n', 's', 'r', 'h', 'l', 'd', 'u', 'c', 'm', 'g', 'y', 'p', 'f', 'w', 'b', '.', 'k', 'v', ',', 'I', 'T', 'S', 'A', 'C', 'W', '-', '/', '0', '?', 'M', 'x', 'j', 'P', 'H', 'B', 'N', 'D', 'E', '1', ':', 'R', 'F', 'O', '2', 'z', 'G', 'L', 'U', '*', '!', 'Y', 'J', 'K', '5', ')', '(', 'q', '3', '4', ';', '9', '8', '6', '7', 'V', '_', '$', ']', '[', '%', '=', '^', '~', 'Z', '&', 'Q', 'X', '#', '+', '\\', '|', '@', '`', '{', '}']
['<unk>', '<pad>', '<BOS>', '<EOS>', ' ', 'e', 't', 'o', 'a', 'i', 'n', 's', 'r', 'h', 'l', 'd', 'u', 'c', 'm', 'y', 'g', 'w', 'p', 'f', '.', 'b', 'k', 'v', 'I', ',', 'T', 'A', 'S', '/', '?', 'j', 'W', '-', 'x', 'H', 'C', '0', 'M', 'N', '!', 'B', 'E', 'O', 'D', 'P', 'R', '1', 'Y', ':', '*', 'F', 'z', 'L', 'G', '2', 'U', ')', '(', 'q', ';', 'J', '5', '3', 'K', '4', '9', '_', '8', '6', '7', 'V', ']', '[', '^', '$', '%', '=', '~', '&', 'Q', 'Z', 'X', '#', '\\', '+', '@', '|', '`', '}', '{']
Input Vocab Si

In [0]:
class ChatBot(nn.Module):
    def __init__(self, 
                 context_vocab_size,  
                 reply_vocab_size, 
                 encoder_hidden_size = 100,
                 generator_hidden_size = 100, 
                 encoder_layers = 1, 
                 generator_layers = 1):
        
        super(ChatBot, self).__init__()
        
        self.encoder_layers = encoder_layers;
        self.generator_layers =generator_layers;
        self.encoder_hidden_size = encoder_hidden_size;
        self.generator_hidden_size = generator_hidden_size;
        
        # >>> Encoder
        self.context_ident = torch.eye(context_vocab_size)
        self.encode_rnn = nn.LSTM(context_vocab_size, encoder_hidden_size, encoder_layers, batch_first=True)
        
        # >>> Generator
        self.reply_ident = torch.eye(reply_vocab_size)
        self.decode_rnn = nn.LSTM(reply_vocab_size, generator_hidden_size, generator_layers, batch_first=True)
        self.fcnn = nn.Linear(generator_hidden_size, reply_vocab_size)
        
    def forward(self, context, response, hidden=None):
        
        # >>> Encoder
        context_tensor = self.context_ident[context] # Type: batch.context[0] | Size: (batch size, sequence size)
        encode_out, encode_last_hidden = self.encode_rnn(context_tensor)
        
        # >>> Generator
        reply_tensor = self.reply_ident[response] #Type: batch.reply
        if(hidden == None):
            gen_out, gen_last_hidden = self.decode_rnn(reply_tensor, encode_last_hidden)
        else:
            gen_out, gen_last_hidden = self.decode_rnn(reply_tensor, hidden)
        out = self.fcnn(gen_out)
        return out, gen_last_hidden

In [0]:
model = ChatBot(input_vocab_size, reply_vocab_size, encoder_layers = 2, generator_layers = 2)
model = model.cuda();
model.context_ident = model.context_ident.cuda();
model.reply_ident = model.reply_ident.cuda();

In [0]:
len(dataset)

1321762

In [0]:
def train_fcn(model, data, reply_vocab_size, batch_size=1, num_epochs=1, lr=0.001, print_every=100):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    it = 0
    
    data_iter = torchtext.data.BucketIterator(data,
                                              batch_size=batch_size,
                                              sort_key=lambda x: len(x.context),
                                              sort_within_batch=True)
    
    iters, losses = [], [];
    
    
    for e in range(num_epochs):
        torch.save(model.state_dict(), '/content/gdrive/My Drive/Chatbot/MM_WEIGHTS_'+str(e))
        print("Epoch : ", e);
        # get training set
        avg_loss = 0
        for batch in data_iter:
            optimizer.zero_grad();
            inp = batch.context[0] # BOS + EOS
            inp = inp.cuda()
            reply = batch.reply[:, :-1]
            reply = reply.cuda()
            
            out, hidden = model(inp, reply) # no eos
            out_reshaped = out.reshape(-1,reply_vocab_size)
            target = (batch.reply[:,1:]).reshape(-1) # no bos
            target = target.cuda()
            #print(batch.reply[:,1:])
            loss = criterion(out_reshaped, target)
            
            loss.backward();
            optimizer.step();
            
            avg_loss += loss
            it += 1 # increment iteration count
            iters.append(it)
            losses.append(float(loss)/batch_size)
            if it % print_every == 0:
                print("[Iter %d] Loss %f" % (it+1, float(avg_loss/print_every)))
                avg_loss = 0
        torch.save(model.state_dict(), '/content/gdrive/My Drive/Chatbot/MM_WEIGHTS_'+str(e))
    
    np.savetxt("{}_train_loss.csv".format('LSTM'), losses);

In [0]:
train_fcn(model, train, reply_vocab_size, batch_size =512, num_epochs = 50)

Epoch :  0
[Iter 101] Loss 1.661044
[Iter 201] Loss 0.797015
[Iter 301] Loss 0.760498
[Iter 401] Loss 0.749910
[Iter 501] Loss 0.743037
[Iter 601] Loss 0.715566
[Iter 701] Loss 0.688625
[Iter 801] Loss 0.660529
[Iter 901] Loss 0.624745
[Iter 1001] Loss 0.602346
[Iter 1101] Loss 0.590690
[Iter 1201] Loss 0.571602
[Iter 1301] Loss 0.560419
[Iter 1401] Loss 0.547505
[Iter 1501] Loss 0.536420
[Iter 1601] Loss 0.527483
[Iter 1701] Loss 0.518374
[Iter 1801] Loss 0.507543
[Iter 1901] Loss 0.500341
[Iter 2001] Loss 0.495979
[Iter 2101] Loss 0.486021
[Iter 2201] Loss 0.481268
[Iter 2301] Loss 0.473040
[Iter 2401] Loss 0.468271
[Iter 2501] Loss 0.463335
Epoch :  1
[Iter 2601] Loss 0.080763
[Iter 2701] Loss 0.459426
[Iter 2801] Loss 0.441515
[Iter 2901] Loss 0.446407
[Iter 3001] Loss 0.439496
[Iter 3101] Loss 0.439194
[Iter 3201] Loss 0.439778
[Iter 3301] Loss 0.429015
[Iter 3401] Loss 0.419408
[Iter 3501] Loss 0.422324
[Iter 3601] Loss 0.424536
[Iter 3701] Loss 0.413139
[Iter 3801] Loss 0.420559

KeyboardInterrupt: ignored

In [0]:
train_loss = np.loadtxt("{}_train_loss.csv".format('LSTM'))
plt.title("Train Loss");
plt.plot(train_loss, label="Train");
plt.xlabel("Iters");
plt.ylabel("Loss");
plt.legend(loc='best');
plt.show();

In [0]:
def sample_sequence(model, context, text_field, label_field, max_len=100, temperature=0.1):
    generated_sequence = ""
    
    data_inp = ["<BOS>"] + list(context) + ["<EOS>"];
    data_inp_indices = [text_field.vocab.stoi[ch] for ch in data_inp];
    data_inp_tensor = torch.Tensor(data_inp_indices).long().unsqueeze(0);
    
    inp = torch.Tensor([label_field.vocab.stoi["<BOS>"]]).long()
    hidden = None;
    for p in range(max_len):
        #print(inp)
        output, hidden = model(data_inp_tensor, inp.unsqueeze(0), hidden)
        #print(output)
        #output = F.softmax(output, dim=2)
        #print(torch.argmax(output, dim=2))
        # Sample from the network as a multinomial distribution
        output_dist = output.data.view(-1).div(temperature).exp()
        #print(output_dist)
        top_i = int(torch.multinomial(output_dist, 1)[0])
        # Add predicted character to string and use as next input
        predicted_char = label_field.vocab.itos[top_i]
        
        if predicted_char == "<EOS>":
            break
        generated_sequence += predicted_char       
        inp = torch.Tensor([top_i]).long()
    return generated_sequence

In [0]:
def sample_sequence2(model, context, text_field, label_field, max_len=100, temperature=0.1):
    generated_sequence = ""
    
    data_inp = ["<BOS>"] + list(context) + ["<EOS>"];
    data_inp_indices = [text_field.vocab.stoi[ch] for ch in data_inp];
    data_inp_tensor = torch.Tensor(data_inp_indices).long().unsqueeze(0);
    
    inp = torch.Tensor([label_field.vocab.stoi["<BOS>"]]).long()
    hidden = None;
    for p in range(max_len):
        #print(inp)
        output, hidden = model(data_inp_tensor, inp.unsqueeze(0), hidden)
        #print(output)
        #output = F.softmax(output, dim=2)
        #print(torch.argmax(output, dim=2))
        # Sample from the network as a multinomial distribution
        output = torch.reshape(output, (1, -1));
        output_dist = F.softmax(output, dim=1)
        #print(output_dist.shape)
        #print(output_dist)
        top_i = int(torch.argmax(output_dist, dim=1))
        # Add predicted character to string and use as next input
        predicted_char = label_field.vocab.itos[top_i]
        
        if predicted_char == "<EOS>":
            break
        generated_sequence += predicted_char       
        inp = torch.Tensor([top_i]).long()
    return generated_sequence

In [0]:
test_model = ChatBot(input_vocab_size, reply_vocab_size, encoder_layers = 2, generator_layers = 2)
test_model.load_state_dict(torch.load('/content/gdrive/My Drive/Chatbot/MM_WEIGHTS_11'))
test_model.eval()

ChatBot(
  (encode_rnn): LSTM(95, 100, num_layers=2, batch_first=True)
  (decode_rnn): LSTM(95, 100, num_layers=2, batch_first=True)
  (fcnn): Linear(in_features=100, out_features=95, bias=True)
)

In [0]:
output1 = sample_sequence(test_model, "fcc or congress?", text_field, label_field, temperature = 0.3)
output2 = sample_sequence2(test_model, "fcc or congress?", text_field, label_field, temperature = 0)
print(output1)
print(output2)

I dont know what they are able to start the same thing they dont have to be a service of the most pe
I would have been a big part of the police of the police of the police of the police of the police o
